# Transformer Model 2 (Model_3 overall): Incomplete/Dropped 

In [ ]:
!pip install transformers datasets --upgrade

testing nothing really

In [ ]:
!pip uninstall -y transformers
!pip uninstall -y huggingface_hub
!pip uninstall -y tokenizers

In [ ]:
!pip install transformers==4.41.1

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import pandas as pd
import numpy as np

In [ ]:
# Load your data (already cleaned, assumed loaded as 'train')
df = train[['Headline', 'Label']].copy()
df = df.rename(columns={'Headline': 'text', 'Label': 'label'})

# Convert to Huggingface Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
model_name = "jy46604790/Fake-News-Bert-Detect"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:
#Define Metrics

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    predictions, labels = pred
    preds = predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1} 

In [ ]:
# Set up Training Arguments:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

In [ ]:
# Initialize the trainer (and train):

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Model_4: Random Forest with TF-IDF parameters for preprocessing 
- By passing pre-processing parameters to TF-IDF, we can reduce the code for a simple classification task


In [ ]:
# Import Data
import pandas as pd

train = pd.read_csv('/Users/iz/Desktop/project-3-nlp/dataset/training_data.csv', sep="\t", header=None, names=['Label', 'Headline'])
test = pd.read_csv('/Users/iz/Desktop/project-3-nlp/dataset/testing_data.csv', sep="\t", header=None, names=['Label', 'Headline'])

In [ ]:
# Split Data (20% val from train set)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train['Headline'], train['Label'], test_size=0.2, random_state=42, stratify=train['Label']
)

In [ ]:
# Create TF-IDF vectorizer (with pre-processing parameters)

from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words="english")

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# Initalize (train and fit) the RandomForest Model with the below parameters:

from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier (n_estimators=200,random_state=42, n_jobs=-1) # num estimators, random state, n_jobs 

model.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:
# Evaluate (use val split data-unseen data to provide an estimate of how the model will perform)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

val_preds = model.predict(X_test_tfidf)

print("Confusion Matrix:")
print(confusion_matrix(y_test, val_preds))

print("\nClassification Report:")
print(classification_report(y_test, val_preds))

print("\nAccuracy Score:")
print(accuracy_score(y_test, val_preds))

Confusion Matrix:
[[3218  297]
 [ 268 3048]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3515
           1       0.91      0.92      0.92      3316

    accuracy                           0.92      6831
   macro avg       0.92      0.92      0.92      6831
weighted avg       0.92      0.92      0.92      6831


Accuracy Score:
0.9172888303323086


In [12]:
# Transform the actual test set
test_tfidf = vectorizer.transform(test['Headline'])

# Predict on the actual test data
test_preds = model.predict(test_tfidf)

# Replace the placeholder 'Label' values with predictions
test['Label'] = test_preds

# Save the results to CSV (same format as original)
test.to_csv('testing_data_predictions_RandomForest.csv', index=False, header=False)
